In [14]:
import pandas as pd
from shapely import wkt
import json

In [18]:
# Load the Excel file
file_path = '/Users/fer/pCloud Drive/2_STORED/0_GEODATABASE_tables/3_UNDERWATER_VIDEO_TRACKS_table.xlsx'
data = pd.read_excel(file_path)

# Select relevant columns for conversion to JSON
relevant_columns = ['dwc:eventID', 'dwc:eventDate', 'dwc:institutionCode', 'dwc:footprintWKT', 'url', 'thumbnail_url','contact']

# Ensure the columns are in the correct format for Leaflet
data_filtered = data[relevant_columns].copy()

# Rename columns to match the JSON structure
data_filtered.rename(columns={
    'dwc:eventID' : 'eventID', 
    'dwc:eventDate' : 'eventDate',
    'dwc:institutionCode': 'institutionCode',
}, inplace=True)

In [19]:
import json
import re
import pandas as pd

# Function to convert a single WKT string into a polyline
def convert_wkt_to_polyline(wkt):
    # Remove the "LINESTRING" part and the parentheses
    coordinates_string = re.sub(r"LINESTRING\s|\(|\)", "", wkt).strip()
    
    # Split the coordinates string into pairs
    coordinates_pairs = coordinates_string.split(",")
    
    # Convert each coordinate pair into a [latitude, longitude] format
    coordinates = []
    for pair in coordinates_pairs:
        lon, lat = map(float, pair.strip().split())
        coordinates.append([lat, lon])  # Leaflet uses [latitude, longitude]
    
    return coordinates

# Assuming data_filtered is your DataFrame and it has a column 'dwc:footprintWKT' containing the WKT strings
data_filtered['line'] = data_filtered['dwc:footprintWKT'].apply(convert_wkt_to_polyline)

print(data_filtered['line'])

0      [[38.90339817412195, 1.4591983268647937], [38....
1      [[38.90287655432948, 1.4604083345644492], [38....
2      [[38.90396821979462, 1.4616800246277342], [38....
3      [[38.90252151050522, 1.4594166627796308], [38....
4      [[38.90101822519989, 1.4544600517407391], [38....
                             ...                        
482    [[38.34625, -0.407768], [38.3462446959473, -0....
483    [[38.329886, -0.438583], [38.3299852544725, -0...
484    [[38.298675, -0.469103], [38.29851754527193, -...
485    [[38.208328, -0.439975], [38.20840648849637, -...
486    [[38.15915, -0.4686], [38.159051807564204, -0....
Name: line, Length: 487, dtype: object


In [20]:
# Drop the original footprintWKT column
data_filtered.drop(columns=['dwc:footprintWKT'], inplace=True)

# Convert the DataFrame to a JSON-like structure
json_data = data_filtered.to_dict(orient='records')

# Prepare the JavaScript variable with data
js_variable = f"var uv_tracks = {json.dumps(json_data, indent=4)};"

# Save to a .js file
output_js_path = '/Users/fer/Documents/PYTHON/15_GEAM_GDB_leaflet/docs/data/3_UNDERWATER_VIDEO_TRACKS_table.js'
with open(output_js_path, 'w') as js_file:
    js_file.write(js_variable)


# Print the path to the JSON file
print(f"JSON file saved to: {output_js_path}")

JSON file saved to: /Users/fer/Documents/PYTHON/15_GEAM_GDB_leaflet/docs/data/3_UNDERWATER_VIDEO_TRACKS_table.js
